# 실전 데이터 탐색 및 분석

## 텍스트 파일 읽기

In [1]:
from google.colab import drive 
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [2]:
cd gdrive

/content/gdrive


In [3]:
cd My Drive

/content/gdrive/My Drive


In [4]:
ls

'1. Backup Files'/                      '6. Corona Virus Dataset'/
'2. Classroom : 빅데이터 인공지능'/     'Colab Notebooks'/
'과제 2. select practice.sql'           'SQL문 추가기능 실습.sql'
'3. Deep learning 개인공부'/             Untitled0.ipynb
'4. Peak Detection and Curve-Fitting'/   Untitled1.ipynb
'5. 빅데이터 인공지능 과정'/


In [5]:
cd 6. Corona Virus Dataset

/content/gdrive/My Drive/6. Corona Virus Dataset


In [6]:
ls

datasets_527325_1205308_Case.csv
datasets_527325_1205308_PatientInfo.csv
datasets_527325_1205308_PatientRoute.csv
datasets_527325_1205308_Policy.csv
datasets_527325_1205308_Region.csv
datasets_527325_1205308_SearchTrend.csv
datasets_527325_1205308_TimeAge.csv
datasets_527325_1205308_Time.csv
datasets_527325_1205308_TimeGender.csv
datasets_527325_1205308_TimeProvince.csv
SeoulFloating.csv
Weather.csv


In [7]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

import json
import requests
import branca
import folium

In [8]:
Case = pd.read_csv("datasets_527325_1205308_Case.csv")

# 감염된 환자들의 신상정보, 주요 활동루트, 
Patient_Info = pd.read_csv("datasets_527325_1205308_PatientInfo.csv")
Patient_Route = pd.read_csv("datasets_527325_1205308_PatientRoute.csv")
Policy = pd.read_csv("datasets_527325_1205308_Policy.csv")

# 지역 데이터 
Region = pd.read_csv("datasets_527325_1205308_Region.csv")
Search_Trend = pd.read_csv("datasets_527325_1205308_SearchTrend.csv")

# 시간관련 정보
Time_Age = pd.read_csv("datasets_527325_1205308_TimeAge.csv")
Time = pd.read_csv("datasets_527325_1205308_Time.csv")
Time_Gender = pd.read_csv("datasets_527325_1205308_TimeGender.csv")
Time_Province = pd.read_csv("datasets_527325_1205308_TimeProvince.csv")

Seoul_Floating = pd.read_csv("SeoulFloating.csv")
Weather = pd.read_csv("Weather.csv")

In [9]:
Case.head(10) 
# 시각화 아이디어 : province, city, infection_case vs confirmed, 좌표 = (lattitude, longitude)

,case_id,province,city,group,infection_case,confirmed,latitude,longitude
0,1000001,Seoul,Yongsan-gu,True,Itaewon Clubs,133.0,37.538621,126.992652
1,1000002,Seoul,Guro-gu,True,Guro-gu Call Center,99.0,37.508163,126.884387
2,1000003,Seoul,Dongdaemun-gu,True,Dongan Church,20.0,37.592888,127.056766
3,1000004,Seoul,Guro-gu,True,Manmin Central Church,41.0,37.481059,126.894343
4,1000005,Seoul,Eunpyeong-gu,True,Eunpyeong St. Mary's Hospital,14.0,37.63369,126.9165
5,1000006,Seoul,Seongdong-gu,True,Seongdong-gu APT,13.0,37.55713,127.0403
6,1000007,Seoul,Jongno-gu,True,Jongno Community Center,10.0,37.57681,127.006
7,1000008,Seoul,Gangnam-gu,True,Samsung Medical Center,7.0,37.48825,127.08559
8,1000009,Seoul,Jung-gu,True,Jung-gu Fashion Company,7.0,37.562405,126.984377
9,1000010,Seoul,Seodaemun-gu,True,Yeonana News Class,4.0,37.558147,126.943799


In [10]:
Patient_Info.head() # 확진자들의 정보

,patient_id,global_num,sex,birth_year,age,country,province,city,disease,infection_case,infection_order,infected_by,contact_number,symptom_onset_date,confirmed_date,released_date,deceased_date,state
0,1000000001,2.0,male,1964,50s,Korea,Seoul,Gangseo-gu,NaN,overseas inflow,1.0,NaN,75,2020-01-22,2020-01-23,2020-02-05,NaN,released
1,1000000002,5.0,male,1987,30s,Korea,Seoul,Jungnang-gu,NaN,overseas inflow,1.0,NaN,31,NaN,2020-01-30,2020-03-02,NaN,released
2,1000000003,6.0,male,1964,50s,Korea,Seoul,Jongno-gu,NaN,contact with patient,2.0,2002000001,17,NaN,2020-01-30,2020-02-19,NaN,released
3,1000000004,7.0,male,1991,20s,Korea,Seoul,Mapo-gu,NaN,overseas inflow,1.0,NaN,9,2020-01-26,2020-01-30,2020-02-15,NaN,released
4,1000000005,9.0,female,1992,20s,Korea,Seoul,Seongbuk-gu,NaN,contact with patient,2.0,1000000002,2,NaN,2020-01-31,2020-02-24,NaN,released


In [11]:
Region.head() # 확진자 지역정보

,code,province,city,latitude,longitude,elementary_school_count,kindergarten_count,university_count,academy_ratio,elderly_population_ratio,elderly_alone_ratio,nursing_home_count
0,10000,Seoul,Seoul,37.566953,126.977977,607,830,48,1.44,15.38,5.8,22739
1,10010,Seoul,Gangnam-gu,37.518421,127.047222,33,38,0,4.18,13.17,4.3,3088
2,10020,Seoul,Gangdong-gu,37.530492,127.123837,27,32,0,1.54,14.55,5.4,1023
3,10030,Seoul,Gangbuk-gu,37.639938,127.025508,14,21,0,0.67,19.49,8.5,628
4,10040,Seoul,Gangseo-gu,37.551166,126.849506,36,56,1,1.17,14.39,5.7,1080


In [12]:
merge = pd.merge(Region, Patient_Info, on = ['city', 'province'])
merge # 지역명, 도시명을 기준으로 합침

,code,province,city,latitude,longitude,elementary_school_count,kindergarten_count,university_count,academy_ratio,elderly_population_ratio,elderly_alone_ratio,nursing_home_count,patient_id,global_num,sex,birth_year,age,country,disease,infection_case,infection_order,infected_by,contact_number,symptom_onset_date,confirmed_date,released_date,deceased_date,state
0,10010,Seoul,Gangnam-gu,37.518421,127.047222,33,38,0,4.18,13.17,4.3,3088,1000000047,1247.0,male,1993,20s,Korea,NaN,Shincheonji Church,NaN,NaN,NaN,NaN,2020-02-26,2020-03-11,NaN,released
1,10010,Seoul,Gangnam-gu,37.518421,127.047222,33,38,0,4.18,13.17,4.3,3088,1000000058,1710.0,female,1990,30s,Korea,NaN,etc,NaN,NaN,NaN,NaN,2020-02-27,NaN,NaN,released
2,10010,Seoul,Gangnam-gu,37.518421,127.047222,33,38,0,4.18,13.17,4.3,3088,1000000064,2468.0,female,1985,30s,Korea,NaN,contact with patient,NaN,1000000047,NaN,2020-02-26,2020-02-29,2020-03-04,NaN,released
3,10010,Seoul,Gangnam-gu,37.518421,127.047222,33,38,0,4.18,13.17,4.3,3088,1000000065,2469.0,male,1987,30s,Korea,NaN,contact with patient,NaN,1000000068,NaN,NaN,2020-02-29,2020-03-14,NaN,released
4,10010,Seoul,Gangnam-gu,37.518421,127.047222,33,38,0,4.18,13.17,4.3,3088,1000000066,2470.0,male,1986,30s,Korea,NaN,etc,NaN,NaN,NaN,NaN,2020-02-29,2020-03-04,NaN,released
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3847,70000,Jeju-do,Jeju-do,33.488936,126.500423,113,123,4,1.53,15.10,6.4,1245,7000000010,NaN,female,NaN,20s,Korea,NaN,overseas inflow,NaN,NaN,18,NaN,2020-04-03,2020-04-21,NaN,released
3848,70000,Jeju-do,Jeju-do,33.488936,126.500423,113,123,4,1.53,15.10,6.4,1245,7000000011,NaN,male,NaN,30s,Korea,NaN,contact with patient,NaN,7000000009,5,NaN,2020-04-03,NaN,NaN,isolated
3849,70000,Jeju-do,Jeju-do,33.488936,126.500423,113,123,4,1.53,15.10,6.4,1245,7000000012,NaN,female,NaN,20s,Korea,NaN,overseas inflow,NaN,NaN,9,NaN,2020-04-03,2020-05-05,NaN,released
3850,70000,Jeju-do,Jeju-do,33.488936,126.500423,113,123,4,1.53,15.10,6.4,1245,7000000013,NaN,female,NaN,10s,China,NaN,overseas inflow,NaN,NaN,6,NaN,2020-04-14,2020-04-26,NaN,released


In [13]:
merge2 = pd.DataFrame(merge[['province', 'city',
                            'latitude', 'longitude',
                            'infection_case']])

merge2 # 원하는 정보 : 지역,도시, 좌표(위도,경도), 감염 원인

,province,city,latitude,longitude,infection_case
0,Seoul,Gangnam-gu,37.518421,127.047222,Shincheonji Church
1,Seoul,Gangnam-gu,37.518421,127.047222,etc
2,Seoul,Gangnam-gu,37.518421,127.047222,contact with patient
3,Seoul,Gangnam-gu,37.518421,127.047222,contact with patient
4,Seoul,Gangnam-gu,37.518421,127.047222,etc
...,...,...,...,...,...
3847,Jeju-do,Jeju-do,33.488936,126.500423,overseas inflow
3848,Jeju-do,Jeju-do,33.488936,126.500423,contact with patient
3849,Jeju-do,Jeju-do,33.488936,126.500423,overseas inflow
3850,Jeju-do,Jeju-do,33.488936,126.500423,overseas inflow


In [14]:
position = merge[['latitude', 'longitude']].dropna() 

position # 좌표정보 null인 것들을 제거

,latitude,longitude
0,37.518421,127.047222
1,37.518421,127.047222
2,37.518421,127.047222
3,37.518421,127.047222
4,37.518421,127.047222
...,...,...
3847,33.488936,126.500423
3848,33.488936,126.500423
3849,33.488936,126.500423
3850,33.488936,126.500423


In [16]:
result_seoul = pd.DataFrame(merge2[merge2.province== 'Seoul'])

result_seoul = result_seoul.reset_index()
result_seoul['infection_case'].value_counts() # 감염경로 별 누적 감염수

overseas inflow                    262
contact with patient               134
Itaewon Clubs                      130
Guro-gu Call Center                 92
etc                                 87
Coupang Logistics Center            20
Dongan Church                       17
Seongdong-gu APT                    13
Eunpyeong St. Mary's Hospital       13
KB Life Insurance                    7
Samsung Medical Center               5
Shincheonji Church                   3
Guri Collective Infection            2
Yeongdeungpo Learning Institute      2
Name: infection_case, dtype: int64

## DataFrame 사용

In [17]:
result_daegu_gyeongbuk = pd.DataFrame(merge2[(merge2.province == 'Daegu') | (merge2.province == 'Gyeongsangbuk-do')])

result_daegu_gyeongbuk = result_daegu_gyeongbuk.reset_index()

# 대구와 경상북도 감염경로
result_daegu_gyeongbuk['infection_case'].value_counts()




etc                                      261
contact with patient                     125
Bonghwa Pureun Nursing Home               31
Shincheonji Church                        31
Cheongdo Daenam Hospital                  20
overseas inflow                           18
Gyeongsan Seorin Nursing Home             15
Gyeongsan Jeil Silver Town                12
Milal Shelter                             11
Gyeongsan Cham Joeun Community Center     10
Pilgrimage to Israel                       2
Name: infection_case, dtype: int64

In [18]:
overseas = Patient_Info[Patient_Info.infection_case == 'overseas inflow'] 

# 감염 경로가 해외 유입인 확진자 데이터 프레임.
overseas

,patient_id,global_num,sex,birth_year,age,country,province,city,disease,infection_case,infection_order,infected_by,contact_number,symptom_onset_date,confirmed_date,released_date,deceased_date,state
0,1000000001,2.0,male,1964,50s,Korea,Seoul,Gangseo-gu,NaN,overseas inflow,1.0,NaN,75,2020-01-22,2020-01-23,2020-02-05,NaN,released
1,1000000002,5.0,male,1987,30s,Korea,Seoul,Jungnang-gu,NaN,overseas inflow,1.0,NaN,31,NaN,2020-01-30,2020-03-02,NaN,released
3,1000000004,7.0,male,1991,20s,Korea,Seoul,Mapo-gu,NaN,overseas inflow,1.0,NaN,9,2020-01-26,2020-01-30,2020-02-15,NaN,released
7,1000000008,13.0,male,1992,20s,Korea,Seoul,etc,NaN,overseas inflow,1.0,NaN,0,NaN,2020-02-02,2020-02-24,NaN,released
8,1000000009,19.0,male,1983,30s,Korea,Seoul,Songpa-gu,NaN,overseas inflow,2.0,NaN,68,NaN,2020-02-05,2020-02-21,NaN,released
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3997,7000000008,9457.0,female,NaN,10s,Korea,Jeju-do,Jeju-do,NaN,overseas inflow,NaN,NaN,25,NaN,2020-03-27,2020-04-12,NaN,released
3998,7000000009,9651.0,female,NaN,20s,Korea,Jeju-do,Jeju-do,NaN,overseas inflow,NaN,NaN,14,NaN,2020-03-29,2020-04-28,NaN,released
3999,7000000010,NaN,female,NaN,20s,Korea,Jeju-do,Jeju-do,NaN,overseas inflow,NaN,NaN,18,NaN,2020-04-03,2020-04-21,NaN,released
4001,7000000012,NaN,female,NaN,20s,Korea,Jeju-do,Jeju-do,NaN,overseas inflow,NaN,NaN,9,NaN,2020-04-03,2020-05-05,NaN,released


In [21]:
infection = set(list(Patient_Info.infection_case))
# 확진자들의 감염경로 종류정보 추출
infection

{'Bonghwa Pureun Nursing Home',
 'Changnyeong Coin Karaoke',
 'Cheongdo Daenam Hospital',
 'Coupang Logistics Center',
 'Dongan Church',
 "Eunpyeong St. Mary's Hospital",
 'Geochang Church',
 'Guri Collective Infection',
 'Guro-gu Call Center',
 'Gyeongsan Cham Joeun Community Center',
 'Gyeongsan Jeil Silver Town',
 'Gyeongsan Seorin Nursing Home',
 'Itaewon Clubs',
 'KB Life Insurance',
 'Milal Shelter',
 'Ministry of Oceans and Fisheries',
 'Onchun Church',
 'Pilgrimage to Israel',
 'River of Grace Community Church',
 'Samsung Medical Center',
 'Seongdong-gu APT',
 'Shincheonji Church',
 'Suyeong-gu Kindergarten',
 'Yeongdeungpo Learning Institute',
 'contact with patient',
 'etc',
 'gym facility in Cheonan',
 'gym facility in Sejong',
 nan,
 'overseas inflow'}

In [22]:
Patient_Info.head()

,patient_id,global_num,sex,birth_year,age,country,province,city,disease,infection_case,infection_order,infected_by,contact_number,symptom_onset_date,confirmed_date,released_date,deceased_date,state
0,1000000001,2.0,male,1964,50s,Korea,Seoul,Gangseo-gu,NaN,overseas inflow,1.0,NaN,75,2020-01-22,2020-01-23,2020-02-05,NaN,released
1,1000000002,5.0,male,1987,30s,Korea,Seoul,Jungnang-gu,NaN,overseas inflow,1.0,NaN,31,NaN,2020-01-30,2020-03-02,NaN,released
2,1000000003,6.0,male,1964,50s,Korea,Seoul,Jongno-gu,NaN,contact with patient,2.0,2002000001,17,NaN,2020-01-30,2020-02-19,NaN,released
3,1000000004,7.0,male,1991,20s,Korea,Seoul,Mapo-gu,NaN,overseas inflow,1.0,NaN,9,2020-01-26,2020-01-30,2020-02-15,NaN,released
4,1000000005,9.0,female,1992,20s,Korea,Seoul,Seongbuk-gu,NaN,contact with patient,2.0,1000000002,2,NaN,2020-01-31,2020-02-24,NaN,released


## 데이터 정제(누락값 처리)

## 데이터 연산 및 필터링

## 데이터 탐색 및 통계

## 데이터 시계열/다차원 분석 및 집계

## 데이터 분석 결과